In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *
from sloppy.metrics import *
from ipywidgets import Layout, IntSlider, FloatLogSlider, FloatSlider, interactive, fixed
import inspect

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\joptic.py:59: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [68]:
import cma

In [3]:
#def LensCavCAD(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, lens_diam=6.35, lens_thick=4., Rlens=5.0):
#    return LensCav(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, lens_diam=lens_diam, lens_thick=lens_thick, Rlens=Rlens)
def LensCavCADpert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, c4=6e-4, c6=0., c8=0., eps=np.zeros((12,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    quart_thick=2.
    quart_dist=1e-6
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    pa = np.array([lens_dist/2.+lens_thick/2.+quart_dist, 0, 0])
    pb = np.array([lens_dist/2.+lens_thick/2.+quart_dist+quart_thick,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    pc = np.array([-lens_dist/2.-lens_thick/2.-quart_dist-quart_thick,0,0])
    pd = np.array([-lens_dist/2.-lens_thick/2.-quart_dist,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])

    ps = np.stack([p0,p1,pa,pb,p2,p3,p4,p5,pc,pd,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
        
    #apply poition perturbations
    ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]    
        
    hi = 12.7
    qi= 7.75
    ng = 1.4537
    coef = np.zeros(9)
    coef[4] = c4
    coef[6] = c6
    coef[8] = c8
    elements = [\
                Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, R=-Rlens, curv='CC', n1=ng),\
                FreeFormInterface(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=lens_diam, n2=ng, coef=coef),\
                Glass(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=lens_diam, n1=ng),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                Mirror(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=hi),\
                Mirror(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=hi),\
                Glass(p=ps[8], n=ns[8], ax=ax_x[8], ay=ax_y[8], Rbasis=Rtr[8], diameter=lens_diam, n2=ng),\
                FreeFormInterface(p=ps[9], n=ns[9], ax=ax_x[9], ay=ax_y[9], Rbasis=Rtr[9], diameter=lens_diam, n1=ng, coef=-coef),\
                CurvedGlass(p=ps[10], n=ns[10], ax=ax_x[10], ay=ax_y[10], Rbasis=Rtr[10], diameter=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[11], n=ns[11], ax=ax_x[11], ay=ax_y[11], Rbasis=Rtr[11], diameter=lens_diam, n1=ng)\
                ]
    return elements

cavfct = LensCavCADpert
parname = 'lens_dist'

In [3]:
def LensCavPert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, eps=np.zeros((8,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])
    ps = np.stack([p0,p1,p2,p3,p4,p5,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
    
    #apply poition perturbations
    ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]   
        
    hi = 12.7
    qi=7.75
    ng = 1.4537
     #negative sign of firstt cuved surface for abcd matrix
    elements = [Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, R=-Rlens, curv='CC', n1=ng),\
                Mirror(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=hi),\
                Mirror(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=hi),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                CurvedGlass(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=lens_diam, n1=ng)]
    return elements

cavfct = LensCavPert
parname = 'lens_dist'

In [4]:
res = degeneracy_length(cavfct, parname, 5e-2, s=1, degmodenum=2)
Ldeg=res.x
print(Ldeg)

21.77801052565133


In [5]:
waists_vs_param(cavfct, parname, 5e-2, s=1, degmodenum=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

21.780501672240803

In [6]:
def get_eigenray_traj(cavfct, cavpars, mu=[0., 0., 0., 0.], Nrt=3, Niter=10, Niter_inner=50, lr=0.03, lr_decay=1.0):
    elements = cavfct(**cavpars)
    sys = RaySystem(elements)
    #ray0 = np.stack([np.atleast_2d(sys.screen.p+np.array([0, 1e-3,0])), np.atleast_2d(sys.screen.n)], axis=0)
    ray0 = sys.screen.eigenvectors_to_rays(mu) #TODO
    reig3, tols = sys.find_eigenray_mpe_dev(ray0, Nrt=Nrt, lr=lr, Niter=Niter_inner, Nmpe=Niter, get_tols=True, lr_decay=lr_decay)
    traj = sys.propagate(reig3, Nrt=Nrt)
    return traj, tols

In [7]:
traj0, tols0 = get_eigenray_traj(cavfct, {parname: Ldeg}, mu=[0, 0, 0, 0], Nrt=1, lr=1e-4, Niter=3)

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\raytracing.py:96: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  rcur = elements[j].propagate(rcur, clip=clip)
c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\raytracing.py:96: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  rcur = elements[j].propagate(rcur, clip=clip)


In [94]:
eps=np.zeros((8,5))
eps[0:2,1] = 10e-3
traj1, tols1 = get_eigenray_traj(cavfct, {parname: Ldeg, 'eps': eps}, mu=[0, 0, 0, 0], Nrt=1, lr=5e-5, Niter=10, Niter_inner=50, lr_decay=0.5)

In [95]:
plt.figure()
plt.semilogy(tols1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
np.linalg.norm((traj0 - traj1)[[0,-1],:,0,:], axis=-1)

array([[0.09877531, 0.0887163 ],
       [0.10496242, 0.0907471 ]])

In [116]:
def get_eigenray_cma(cavfct, cavpars, mu=[0., 0., 0., 0.], Nrt=1, sigma=0.1, verb_disp=100):
    elements = cavfct(**cavpars)
    sys = RaySystem(elements)
    
    def loss(x, sys, Nrt=1):
        rcur = sys.screen.eigenvectors_to_rays(np.asarray(x))
        traj = sys.propagate(rcur, Nrt=Nrt)
        rnew = traj[-1,...]
        return np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()
    
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    es = cma.CMAEvolutionStrategy(mu, sigma)
    es.optimize(loss, args=(sys, Nrt), verb_disp=verb_disp)
    mueig = es.result.xbest
    reig3 = sys.screen.eigenvectors_to_rays(mueig)
    traj = sys.propagate(reig3, Nrt=Nrt)
    return traj, [es.result.fbest,]

In [117]:
traj2, tols2 = get_eigenray_cma(cavfct, {parname: Ldeg, 'eps': eps}, mu=[0, 0, 0, 0], Nrt=1, verb_disp=100)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=977880, Mon Oct  5 19:07:13 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 8.611248318631545e-02 1.0e+00 8.99e-02  8e-02  1e-01 0:00.0
    2     16 1.684427524321790e-01 1.4e+00 8.45e-02  6e-02  9e-02 0:00.0
    3     24 1.371180828529664e-01 1.6e+00 8.55e-02  6e-02  1e-01 0:00.0


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [7]/[] are nan/None and will be set to the median value 0.2008250681512932 (class=CMAEvolutionStrategy method=ask iteration=3)
  ')')


  100    800 4.011597896525037e-05 6.6e+01 5.63e-05  5e-06  1e-05 0:00.3
  159   1272 4.010264721409381e-05 5.6e+01 2.66e-07  2e-09  4e-09 0:00.4
  159   1272 4.010264721409381e-05 5.6e+01 2.66e-07  2e-09  4e-09 0:00.4
termination by {'tolfun': 1e-11}
best f-value = 4.010264721409381e-05
solution = [ 0.06813127  0.03178675  0.02740206 -0.05696504]


In [103]:
try:
    plot.close()
except:
    pass
elements = cavfct(**{parname: Ldeg})
plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)
    
plot_trajs(traj0, plot)
plot_trajs(traj1, plot, color=0x00ff00)
plot_trajs(traj2, plot, color=0xff0000)
plot += k3d.points(traj0[:,0,0,:], color=0xff0000, point_size=0.3)

plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [118]:
def get_perturbation_one(cavfct, cavpars, which, ax, rng, mu=[0, 0, 0, 0], Nrt=3, lr=0.01, Niter=20, relative=True, get_tols=False):
    #get reference trajectory
    traj0, tols0 = get_eigenray_traj(cavfct, cavpars, mu=mu, Nrt=Nrt)
    
    #move one element and find perturbed eigenray trajectory
    N = rng.shape[0]
    alltraj = np.zeros((N,traj0.shape[0],3))
    alltols = np.zeros(N)
    for i, e in enumerate(rng):
        eps = np.zeros((8,5))
        eps[which, ax] += e
        pars = cavpars.copy()
        pars['eps'] = eps
        try:
            #traj1, tols1 = get_eigenray_traj(cavfct, pars, mu=mu, Nrt=Nrt, lr=lr, Niter=Niter, Niter_inner=50, lr_decay=0.5) #get_eigenray_traj(cavfct, pars, mu=mu, Nrt=Nrt, lr=lr)
            traj1, tols1 = get_eigenray_cma(cavfct, pars, mu=mu, Nrt=Nrt, verb_disp=False)
        except:
            alltraj[i,...] = np.nan
            alltols[i] = np.nan
        else:
            alltraj[i,...] = traj1[:,0,0,:]
            alltols[i] = tols1[-1]
        
    if relative:
        if get_tols:
            return np.linalg.norm(traj0[None,:,0,0,:]-alltraj, axis=-1), alltols
        else:
            return np.linalg.norm(traj0[None,:,0,0,:]-alltraj, axis=-1)
    else:
        if get_tols:
            return alltraj, alltols
        else:
            return alltraj

In [152]:
def get_perturbation_ray(cavfct, cavpars, which, ax, rng, mu=[0, 0, 0, 0], Nrt=3, get_tols=False):
    #get reference trajectory
    traj0, tols0 = get_eigenray_traj(cavfct, cavpars, mu=mu, Nrt=Nrt)
    
    #move one element and find perturbed eigenray trajectory
    N = rng.shape[0]
    alltraj = np.zeros((N,traj0.shape[0],2,3))
    alltols = np.zeros(N)
    for i, e in enumerate(rng):
        eps = np.zeros((8,5))
        eps[which, ax] += e
        pars = cavpars.copy()
        pars['eps'] = eps
        try:
            traj1, tols1 = get_eigenray_cma(cavfct, pars, mu=mu, Nrt=Nrt, verb_disp=False)
        except:
            alltraj[i,...] = np.nan
            alltols[i] = np.nan
        else:
            alltraj[i,...] = traj1[:,:,0,:]
            alltols[i] = tols1[-1]
        
    if get_tols:
        return alltraj, alltols
    else:
        return alltraj

In [108]:
rng = np.linspace(0, 5e-3, 50)
devs, tols = get_perturbation_one(cavfct, cavpars={parname: Ldeg}, Nrt=1, which=[0,1], ax=1, rng=rng, lr=5e-5, Niter=5, get_tols=True)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1008176, Mon Oct  5 18:58:35 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.1e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=218)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1002732, Mon Oct  5 18:58:42 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [6]/[] are nan/None and will be set to the median value 0.2842589459951028 (class=CMAEvolutionStrategy method=ask iteration=2)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=953835, Mon Oct  5 18:58:48 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1051422, Mon Oct  5 18:58:52 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1095048, Mon Oct  5 18:58:58 2020)


...

C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [2]/[] are nan/None and will be set to the median value 0.2889507205623614 (class=CMAEvolutionStrategy method=ask )
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=992552, Mon Oct  5 19:02:11 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [1, 4]/[] are nan/None and will be set to the median value 0.25239115040979515 (class=CMAEvolutionStrategy method=ask iteration=1)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1047731, Mon Oct  5 19:02:16 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=983857, Mon Oct  5 19:02:21 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=969252, Mon Oct  5 19:02:25 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [6]/[] are nan/None and will be set to the median value 0.29354333889507556 (class=CMAEvolutionStrategy method=ask )
  ')')


In [110]:
plt.figure()
plt.plot(rng, devs)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
plt.figure()
plt.semilogy(rng, tols)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
plt.figure()
plt.semilogy(tols0)
plt.semilogy(tols1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
rng = np.linspace(0, 20e-3, 100)
traj_anim, tols = get_perturbation_one(cavfct, cavpars={parname: Ldeg}, Nrt=1, which=[0,1], ax=1, rng=rng, lr=5e-5, Niter=5, relative=False, get_tols=True)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=998988, Mon Oct  5 19:07:43 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.1e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=198)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=985670, Mon Oct  5 19:07:50 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [0]/[] are nan/None and will be set to the median value 0.23589833637927599 (class=CMAEvolutionStrategy method=ask )
  ')')
C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [3]/[] are nan/None and will be set to the median value 0.29404883266816007 (class=CMAEvolutionStrategy method=ask iteration=2)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1102285, Mon Oct  5 19:07:54 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=991400, Mon Oct  5 19:08:00 2020)


...

C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [2]/[] are nan/None and will be set to the median value 0.2725136876394876 (class=CMAEvolutionStrategy method=ask iteration=2)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1062223, Mon Oct  5 19:25:37 2020)
(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1019723, Mon Oct  5 19:25:49 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [0]/[] are nan/None and will be set to the median value 0.3713775873510876 (class=CMAEvolutionStrategy method=ask )
  ')')
C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [3]/[] are nan/None and will be set to the median value 0.2553412813543267 (class=CMAEvolutionStrategy method=ask iteration=1)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1091465, Mon Oct  5 19:26:04 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [1]/[] are nan/None and will be set to the median value 0.3191017159585493 (class=CMAEvolutionStrategy method=ask iteration=1)
  ')')
C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [6]/[] are nan/None and will be set to the median value 0.4112921733279037 (class=CMAEvolutionStrategy method=ask iteration=2)
  ')')
C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [4]/[] are nan/None and will be set to the median value 0.3197345584961595 (class=CMAEvolutionStrategy method=ask iteration=3)
  ')')


In [153]:
rng = np.linspace(0, 40e-3, 50)
rays_anim, tols = get_perturbation_ray(cavfct, cavpars={parname: Ldeg}, Nrt=1, which=[0,1], ax=1, rng=rng, get_tols=True)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1106912, Mon Oct  5 20:45:58 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: 
        geno-pheno transformation introduced based on the
        current covariance matrix with condition 1.3e+12 -> 1.0e+00,
        injected solutions become "invalid" in this iteration (class=CMAEvolutionStrategy method=alleviate_conditioning iteration=197)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1001967, Mon Oct  5 20:46:05 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [4]/[] are nan/None and will be set to the median value 0.5055692207451318 (class=CMAEvolutionStrategy method=ask )
  ')')
C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [0]/[] are nan/None and will be set to the median value 0.2353605394718592 (class=CMAEvolutionStrategy method=ask iteration=2)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1092528, Mon Oct  5 20:46:10 2020)


...

C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [5]/[] are nan/None and will be set to the median value 0.15282350598097166 (class=CMAEvolutionStrategy method=ask iteration=4)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1047692, Mon Oct  5 20:53:28 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [1, 2]/[] are nan/None and will be set to the median value 0.3583002622252594 (class=CMAEvolutionStrategy method=ask iteration=3)
  ')')


(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1079822, Mon Oct  5 20:53:48 2020)


C:\ProgramData\Anaconda3\lib\site-packages\cma\utilities\utils.py:347: UserWarning: function values with index [7]/[] are nan/None and will be set to the median value 0.38118545193828507 (class=CMAEvolutionStrategy method=ask )
  ')')


In [149]:
from joblib import dump

In [150]:
savedict = {'range': rng, 'trajectories': traj_anim}
dump(savedict, 'trajectories_y_20um.p')

['trajectories_y_20um.p']

In [155]:
traj_anim = rays_anim[:,:,0,:]

In [156]:
traj_anim_ts = traj_to_timeseries(traj_anim[:,:,None,:], step=1)

In [157]:
elements = cavfct(**{parname: Ldeg})

In [158]:
plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)

#plot_trajs(traj, plot)
for ts in traj_anim_ts:
    plot += k3d.line(ts, shader='mesh', width=0.05, color=0x00ff00)

plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## Metrics
- Lens curved surface separation
- Lens curved surface AOI
- Lens flat surface AOI
- Displacement / tilt of cavity axis at the input mirror i.e., which modes get excited by a fixed input beam?

In [159]:
traj_anim[0,:,:]

array([[ 0.00000000e+00, -1.38818695e-06,  1.88643777e-07],
       [ 8.88900526e+00, -2.79494531e-06, -1.01633826e-05],
       [ 1.28890053e+01, -3.23040859e-06, -1.33678614e-05],
       [ 2.39999956e+01, -1.73188008e-06, -1.28299558e-05],
       [ 9.43979638e+00,  1.90000111e+01,  1.16571835e+01],
       [-9.43979989e+00,  1.90000117e+01, -1.16571830e+01],
       [-2.39999972e+01,  1.75467489e-06,  1.28268606e-05],
       [-1.28890053e+01,  4.54031844e-07,  1.37451517e-05],
       [-8.88900526e+00,  1.85692880e-08,  1.05406721e-05],
       [ 0.00000000e+00, -1.38818672e-06,  1.88643433e-07]])

In [160]:
lens_separation = np.linalg.norm(traj_anim[:,2,:] - traj_anim[:,-3,:], axis=-1)
plt.figure()
plt.plot(rng*1e3, lens_separation)
plt.xlabel('Lens displacement [um]')
plt.ylabel('Curved surface separation [mm]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [161]:
input_displace = np.linalg.norm(traj_anim[:,-4,:] - traj0[-4,0,0,:], axis=-1)
plt.figure()
plt.plot(rng*1e3, input_displace)
plt.xlabel('Lens displacement [um]')
plt.ylabel('Ray displacement @ Input mirror  [mm]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [168]:
r = rays_anim[0,0,:,:]

In [178]:
aoi_flat1 = np.arccos(np.dot(rays_anim[:,0,1,:], -elements[0].n))
plt.figure()
plt.plot(rng*1e3, np.rad2deg(aoi_flat1))
plt.xlabel('Lens displacement [um]')
plt.ylabel('AOI @ flat surface 1 [deg]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [182]:
aoi_flat2 = np.arccos(np.dot(rays_anim[:,-2,1,:], -elements[7].n)) # -3: after curved glass, 7: last flat glass
plt.figure()
plt.plot(rng*1e3, np.rad2deg(aoi_flat2))
plt.xlabel('Lens displacement [um]')
plt.ylabel('AOI @ flat surface 2 [deg]')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
elements[1].jopt.otype

5

In [191]:
n = elements[1].jopt.cp - rays_anim[0,2,0,:]
n = n/np.linalg.norm(n)

In [192]:
np.arccos(np.dot(rays_anim[:,2,1,:], -n)) #TODO catch: curved lens poition also moves!!! Find a good way to do that...

array([2.83173036e-06, 3.36286867e-03, 4.23675957e-03, 4.84989790e-03,
       5.33810500e-03, 5.75044785e-03, 6.11094147e-03, 6.43335215e-03,
       6.72639024e-03, 6.99596042e-03, 7.24626948e-03, 7.48043447e-03,
       7.70083624e-03, 7.90933857e-03, 8.10743258e-03, 8.29633059e-03,
       8.47703403e-03, 8.65038007e-03, 8.81707797e-03, 8.97773235e-03,
       9.13286504e-03, 9.28292899e-03, 9.42832172e-03, 9.56939111e-03,
       9.70644666e-03, 9.83976467e-03, 9.96958991e-03, 1.00961444e-02,
       1.02196278e-02, 1.03402207e-02, 1.04580884e-02, 1.05733797e-02,
       1.06862326e-02, 1.07967729e-02, 1.09051161e-02, 1.10113693e-02,
       1.11156310e-02, 1.12179929e-02, 1.13185401e-02, 1.14173509e-02,
       1.15144999e-02, 1.16100550e-02, 1.17040811e-02, 1.17966388e-02,
       1.18877841e-02, 1.19775707e-02, 1.20660482e-02, 1.21532639e-02,
       1.22392625e-02, 1.23240861e-02])

## Optimization Test

In [12]:
from scipy.optimize import minimize, dual_annealing, basinhopping

In [13]:
eps=np.zeros((8,5))
eps[0:2,1] = 1e-3
elements = cavfct(**{parname: Ldeg, 'eps': eps})
sys = RaySystem(elements)

In [14]:
x = np.array([0., 0., 0., 0.])
rcur = sys.screen.eigenvectors_to_rays(x)
traj = sys.propagate(rcur, Nrt=1)
rnew = traj[-1,...]

In [62]:
rnew

array([[[ 3.38020349e-08, -1.00932977e-04,  1.43981782e-04]],

       [[ 9.99999891e-01,  2.67781338e-04, -3.81992439e-04]]])

In [35]:
np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()

0.0010545920467091205

In [85]:
np.linalg.norm(rcur-rnew)

0.0007566473753843225

In [49]:
def loss(x, sys, Nrt=1):
    rcur = sys.screen.eigenvectors_to_rays(np.asarray(x))
    traj = sys.propagate(rcur, Nrt=Nrt)
    rnew = traj[-1,...]
    return np.linalg.norm(rnew[:,0,:]-rcur[:,0,:], axis=-1).sum()

In [50]:
loss(x, sys)

0.0027953745195005058

In [59]:
a = 5e-3
N = 100
xx = np.linspace(-a, a, 50)
x, y = np.meshgrid(xx, xx)
losses = np.empty_like(x)

In [60]:
for i in range(len(xx)):
    for j in range(len(xx)):
        losses[i, j] = loss(np.array([5e-4, 0., x[i,j], y[i,j]]), sys)

In [61]:
plt.figure()
plt.pcolormesh(x, y, losses)
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
res = minimize(loss, x0=np.array([0., 0., 0., 0.]), args=(sys,), method='L-BFGS-B', bounds=[(-1, 1), (-1, 1), (-0.5, 0.5), (-0.5, 0.5)]) 

In [53]:
#res = minimize(loss, x0=(0., 0., 0., 0.), args=(sys,), method='trust-constr', bounds=[(-1, 1), (-1, 1), (-0.5, 0.5), (-0.5, 0.5)]) 

In [63]:
res = minimize(loss, x0=np.array([0., 0., -5e-4, 1e-4]), args=(sys,), method='BFGS')

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optic.py:104: RuntimeWarning: invalid value encountered in sqrt
  s0 = np.stack([mu[:,0], mu[:,1], np.sqrt(1 - (mu[:,0])**2 - (mu[:,1])**2)], axis=1) #create vector normal to screen
c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optic.py:104: RuntimeWarning: invalid value encountered in sqrt
  s0 = np.stack([mu[:,0], mu[:,1], np.sqrt(1 - (mu[:,0])**2 - (mu[:,1])**2)], axis=1) #create vector normal to screen
c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\optic.py:104: RuntimeWarning: invalid value encountered in sqrt
  s0 = np.stack([mu[:,0], mu[:,1], np.sqrt(1 - (mu[:,0])**2 - (mu[:,1])**2)], axis=1) #create vector normal to screen


In [64]:
res

      fun: nan
 hess_inv: array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])
      jac: array([nan, nan, nan, nan])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 672
      nit: 1
     njev: 112
   status: 2
  success: False
        x: array([-310.362662  , -587.8101473 ,  700.66896752, -369.95790256])

In [65]:
lw = [-0.5] * 4
up = [0.5] * 4
ret = dual_annealing(loss, bounds=list(zip(lw, up)), args=(sys, ), seed=1234, maxiter=10**4)

In [66]:
ret

     fun: nan
 message: ['Maximum number of iteration reached']
    nfev: 351894
    nhev: 0
     nit: 10000
    njev: 0
  status: 0
 success: True
       x: array([ 0.27997581, -0.22740739, -0.22353574,  0.30187218])

In [75]:
es = cma.CMAEvolutionStrategy(4 * [0], 0.1)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=1010753, Mon Oct  5 18:44:28 2020)


In [77]:
es.optimize(loss, args=(sys,), verb_disp=100)

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 1.500504133740355e-01 1.0e+00 8.00e-02  7e-02  8e-02 0:00.0
    2     16 1.372318804411779e-01 1.2e+00 7.18e-02  6e-02  7e-02 0:00.0
    3     24 7.923790957990179e-02 1.3e+00 6.84e-02  6e-02  7e-02 0:00.0
  100    800 3.363774002143986e-06 1.5e+02 2.44e-04  4e-05  7e-05 0:00.2
  172   1376 1.853903724680585e-06 3.1e+02 1.51e-07  2e-09  4e-09 0:00.4
  172   1376 1.853903724680585e-06 3.1e+02 1.51e-07  2e-09  4e-09 0:00.4
termination by {'tolfun': 1e-11}
best f-value = 1.8539037212523853e-06
solution = [ 0.0316697   0.01513088  0.01276856 -0.02654883]


In [84]:
mueig = es.result.xbest
reig2 = sys.screen.eigenvectors_to_rays(mueig)

In [113]:
xopt, es = cma.fmin2(loss, 4 * [0], 0.1, args=(sys,), n_jobs=8)

TypeError: fmin() got an unexpected keyword argument 'n_jobs'

In [87]:
traj2 = sys.propagate(reig2)

In [89]:
try:
    plot.close()
except:
    pass
elements = cavfct(**{parname: Ldeg})
plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements):
    plot += el.plot(opacity=0.4)
    
plot_trajs(traj0, plot)
plot_trajs(traj1, plot, color=0x00ff00)
plot_trajs(traj1, plot, color=0xff0000)

plot += k3d.points(traj0[:,0,0,:], color=0xff0000, point_size=0.3)

plot.display()

Output()

In [151]:
traj2.shape

(10, 2, 1, 3)

In [46]:
def rays_interaction_factory_pert(cavfct, cavpars, rmax=6.):
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(11,6.6), sharex='row', sharey='row')
    ms = 8.
    lines = [a.scatter([], [], c=[], cmap='jet', s=ms) for a in ax.flatten()]
    ax[0,0].set_title('Raytracing')
    ax[0,1].set_title('ABCD')
    ax[0,2].set_title('ABCD (Extracted)')
    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')

    def makeidx(hit_m):
        return np.arange(hit_m.shape[0])
    
    magnitude = lambda x: 1. if x==0. else int(np.floor(np.log10(x)))
    
    def update(ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        #kwargs.update({parname: Ldeg+dl*1e-1})
        elements = cavfct(**kwargs)
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0] #fix for now since waist is not at 0 anymore!
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_scrs = sys.screen.s_to_screen_coords(traj_hit[:,1,0,:])

        lines[0].set_offsets(hit_scr[:,:])
        lines[0].set_array(makeidx(hit_scr))
        lines[3].set_offsets(hit_scrs[:,:])
        lines[3].set_array(makeidx(hit_scrs))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lines[1].set_offsets(hit_m[:,0:2])
        lines[1].set_array(makeidx(hit_m))
        lines[4].set_offsets(hit_m[:,2:])
        lines[4].set_array(makeidx(hit_m))
        
        #abcd_fd = sys.extract_ABCD(epsr=1e-3, epss=1e-3, Nrt=1) 
        #hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        #lines[2].set_offsets(hit_fd[:,0:2])
        #lines[2].set_array(makeidx(hit_fd))
        #lines[5].set_offsets(hit_fd[:,2:])
        #lines[5].set_array(makeidx(hit_fd))
        
        for l in lines:
            l.set_clim(vmin=0, vmax=Nrt)
            
        ax[0,0].set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax[0,0].set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        ax[1,0].set_xlim(1.5*np.nanmin(hit_scrs[:,0]), 1.5*np.nanmax(hit_scrs[:,0]))
        ax[1,0].set_ylim(1.5*np.nanmin(hit_scrs[:,1]), 1.5*np.nanmax(hit_scrs[:,1]))
        fig.canvas.draw_idle()

    lo = Layout(width='80%', height='30px')

    sliders = {v.name: FloatSlider(value=v.default, min=v.default*0.1, max=v.default*1.5, step=10**(magnitude(v.default)-3), readout_format='.2e', layout=lo) for v in inspect.signature(cavfct).parameters.values() if v.name not in cavpars.keys()}
    ars = FloatSlider(value=0.1, min=0., max=rmax, step=1e-2, readout_format='.3f', layout=lo)
    brs = FloatSlider(value=0.1, min=0., max=rmax, step=1e-2, readout_format='.3f', layout=lo)
    aps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
    bps = FloatSlider(value=0., min=0., max=1*np.pi, step=1e-2, readout_format='.3f', layout=lo)
    Nrts = IntSlider(value=100, min=100, max=2000, step=100, layout=lo)
    raysliders = {'ar': ars, 'br': brs, 'ap': aps, 'bp': bps, 'Nrt': Nrts}
    sliders.update(**raysliders)
    cavparsfixed = {k: fixed(v) for k, v in cavpars.items()}
    sliders.update(**cavparsfixed)
    #print(sliders)
    return interactive(update, **sliders)

In [47]:
eps = np.zeros((12,5))
eps[3,3] += 1e-1
rays_interaction_factory_pert(cavfct, cavpars={parname: Ldeg, 'eps': eps}, rmax=10.)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='ar', layout=Layout(height='30px', width='80%'), max=…

In [34]:
metrics_interaction_factory(cavfct, cavpars={parname: Ldeg, 'c4': 6e-4, 'c6': 0, 'c8': 0}, rmax=12., Nplt=100, slrng=(0.8, 1.2), degmodenum=1, a=8., b=0., s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'lens_dist': 1, 'c4': -4, 'c6': 1.0, 'c8': 1.0}


interactive(children=(FloatSlider(value=8.0, description='a', layout=Layout(height='30px', width='80%'), max=1…

In [35]:
dct = {"a": 1, "b": 2}

In [36]:
for k,v in dct.items():
    print(k,v)

a 1
b 2


In [17]:
res = degeneracy_length(cavfct, parname, 1e-2)
Ldeg=res.x
sys = RaySystem(cavfct(lens_dist=Ldeg))

try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(sys.elements):
    plot_element_ax(el, plot, length=5.)
plot.display()

Output()

In [19]:
system = sys.abcd
print(system.Ltot)
system.waist_at(0)
mu1, mu2 = system.q

131.24995092745596


In [20]:
system.waist_at(0.)

array([0.0182912, 0.0182912])

In [21]:
mu = 1e-1*np.real(mu1) + 1e-3*np.real(mu2) 
ray = sys.screen.eigenvectors_to_rays(mu)

traj = sys.propagate(ray, Nrt=30)

In [22]:
screen = Screen(p=sys.screen.p, n=sys.screen.n, ax=sys.screen.ax, ay=sys.screen.ay, diameter=3.)

In [23]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements[:-1]):
    plot += el.plot(opacity=0.4)

plot += screen.plot(opacity=0.4)

plot_trajs(traj, plot, shader='mesh', width=0.02, color=0xc61920)
plot.display()

Output()